In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
import requests
import time
import sys
import os
from scipy.stats import linregress
import scipy.stats as st
from pprint import pprint
sys.path.append('../')
from api_keys import sportsio_key
print(sportsio_key)

fd22e44264ab4828a2cfe860923da7f0


In [2]:
#Import the rating and salary files

#Player Stats
stats_df = pd.read_csv('Resources/Obsolete/100_stats.csv')

#Player Salaries
salaries_dict= pd.read_excel('Resources/Obsolete/population_salaries_by_season.xlsx',sheet_name=None)


In [5]:
#Convert dict to a data frame

salaries_df=pd.DataFrame(salaries_dict["salaries"])
stats_df.head()

,Player,season_year,Tm,Age,Pos,G,GS,Wins,Losses,Ties,...,Y/G,Rate,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD,QBR
0,Spergon Wynn,2000,CLE,22,QB,7,1,0,1,0,...,23.9,41.2,13.0,89,19.4,1.16,0.49,NaN,NaN,NaN
1,Chad Pennington,2000,NYJ,24,QB,1,0,0,0,0,...,67.0,127.1,1.0,4,16.7,10.50,13.83,NaN,NaN,NaN
2,Chris Redman,2000,BAL,23,QB,2,0,0,0,0,...,9.5,84.0,0.0,0,0.0,6.33,6.33,NaN,NaN,NaN
3,Tom Brady,2000,NWE,23,QB,1,0,0,0,0,...,6.0,42.4,0.0,0,0.0,2.00,2.00,NaN,NaN,NaN
4,Tim Rattay,2000,SFO,23,QB,1,0,0,0,0,...,-4.0,79.2,0.0,0,0.0,-4.00,-4.00,NaN,NaN,NaN


In [4]:
salaries_df.head()

,career_Y,Tom Brady,Drew Brees,Ben Roethlisberger,Philip Rivers,Eli Manning,Matt Ryan,Aaron Rodgers,Mathew Stafford,Carson Palmer,...,Curtis Painter,Sean Mannion,Tom Savage,Marques Tuiasosopo,Pat White,Bryce Petty,Ryan Lindley,Spergon Wynn,John Beck,Drew Henson
0,Y1,205833,627500.0,1502000.0,2580000.0,2244000.0,6600000.0,1150000.0,3100000.0,2500000.0,...,630000.0,591449.0,495146.0,NaN,NaN,573424.0,416327.0,NaN,175000.0,NaN
1,Y2,310833,707500.0,4220250.0,4855000.0,4527926.0,9900000.0,1402500.0,12980000.0,5477242.0,...,NaN,738734.0,408146.0,NaN,NaN,663424.0,506327.0,NaN,NaN,NaN
2,Y3,1087834,2287500.0,4545750.0,5165000.0,7388074.0,12250000.0,1557500.0,6123750.0,8982758.0,...,NaN,886486.0,675146.0,NaN,NaN,753424.0,234706.0,NaN,NaN,NaN
3,Y4,3318750,2367500.0,3127000.0,10775000.0,10046666.0,13250000.0,13952500.0,8935000.0,12980000.0,...,NaN,1034227.0,765146.0,NaN,NaN,NaN,38824.0,NaN,NaN,NaN
4,Y5,5058750,8078000.0,7970000.0,7625000.0,12916666.0,13500000.0,9652500.0,17820000.0,13480000.0,...,NaN,735000.0,207353.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Changed the rows//columns so columns represent year and rows are Player name
salaries1_df=salaries_df.transpose()

In [18]:
salaries1_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
career_Y,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,...,Y12,Y13,Y14,Y15,Y16,Y17,Y18,Y19,Y20,Y21
Tom Brady,205833,310833,1087834,3318750,5058750,8423750,13823750,7340000,14620000,14620000,...,13200000,8000000,13800000,14800000,14000000,13764706,14000000,22000000,21500000,25000000
Drew Brees,627500.0,707500.0,2287500.0,2367500.0,8078000.0,6000000.0,7000000.0,9000000.0,10660400.0,12358400.0,...,10400000.0,17400000.0,18400000.0,23800000.0,17250000.0,19000000.0,24000000.0,22700000.0,23650000.0,NaN
Ben Roethlisberger,1502000.0,4220250.0,4545750.0,3127000.0,7970000.0,13220000.0,10355882.0,11320000.0,8895000.0,13595000.0,...,17245000.0,23950000.0,18200000.0,23200000.0,26200000.0,23750000.0,NaN,NaN,NaN,NaN
Philip Rivers,2580000.0,4855000.0,5165000.0,10775000.0,7625000.0,11535000.0,11710000.0,13510000.0,15310000.0,13776666.0,...,21166668.0,16500000.0,18000000.0,22000000.0,23000000.0,25000000.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bryce Petty,573424.0,663424.0,753424.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ryan Lindley,416327.0,506327.0,234706.0,38824.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Spergon Wynn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
John Beck,175000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#Drop the first row since it is not relevant. We assume the first data is when they first started to play
salaries1_df=salaries1_df.iloc[1:,:]

In [22]:
#Reset Index so we have Player names as one of the column. This may not be needed?
salaries1_df=salaries1_df.reset_index()


In [23]:
#Changed Index to Player
salaries1_df=salaries1_df.rename(columns={"index": "Player"})
salaries1_df

,Player,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,Tom Brady,205833,310833,1087834,3318750,5058750,8423750,13823750,7340000,14620000,...,13200000,8000000,13800000,14800000,14000000,13764706,14000000,22000000,21500000,25000000
1,Drew Brees,627500.0,707500.0,2287500.0,2367500.0,8078000.0,6000000.0,7000000.0,9000000.0,10660400.0,...,10400000.0,17400000.0,18400000.0,23800000.0,17250000.0,19000000.0,24000000.0,22700000.0,23650000.0,NaN
2,Ben Roethlisberger,1502000.0,4220250.0,4545750.0,3127000.0,7970000.0,13220000.0,10355882.0,11320000.0,8895000.0,...,17245000.0,23950000.0,18200000.0,23200000.0,26200000.0,23750000.0,NaN,NaN,NaN,NaN
3,Philip Rivers,2580000.0,4855000.0,5165000.0,10775000.0,7625000.0,11535000.0,11710000.0,13510000.0,15310000.0,...,21166668.0,16500000.0,18000000.0,22000000.0,23000000.0,25000000.0,NaN,NaN,NaN,NaN
4,Eli Manning,2244000.0,4527926.0,7388074.0,10046666.0,12916666.0,13066668.0,12100000.0,14100000.0,9600000.0,...,14450000.0,24200000.0,19700000.0,22200000.0,23200000.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Bryce Petty,573424.0,663424.0,753424.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Ryan Lindley,416327.0,506327.0,234706.0,38824.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Spergon Wynn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,John Beck,175000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#Sort the stats by Seaon year so they are in ascending order, which should align with the salary table
stats_df=stats_df.sort_values(by=['season_year'])

#ADD a new column called Salary
stats_df["Salary"]=""


In [26]:
#len(stats_df1.loc[stats_df1['Player'] == g[0]])
#Loop through the 100 players and copy theiur salaries to the stats table oin "Salary column"
#It is not clean way to do things. Will explore more ways to improve it later
i=1
#First row is title
j=0
for player in salaries1_df.iloc[0:,0]:
    i=1
    for row in stats_df.loc[stats_df['Player'] == player].index:
#Debugging only
#        print (f"Player = {player}")
#        print(f"row = {row}")
#        print(f"i={i}")
#        print(f"Salaries = {salaries1_df.iloc[0,i]}")
#        print(f"Stats value = {stats_df.loc[row , 'Salary']}")
#        salary=salaries1_df.iloc[j,i]
        stats_df.loc[row , "Salary"] = salaries1_df.iloc[j ,i]
        i=i+1
    j=j+1


In [45]:
stats_df.to_csv("Output/raw.csv", index=False, header=True)
print(len(stats_df))

722
